In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tf_mnist", one_hot=True)

Extracting ./tf_mnist\train-images-idx3-ubyte.gz
Extracting ./tf_mnist\train-labels-idx1-ubyte.gz
Extracting ./tf_mnist\t10k-images-idx3-ubyte.gz
Extracting ./tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-3
training_iters = int(1e5)
batch_size = 128
display_step = 25

In [4]:
n_input = 28
n_steps = 28
n_h1 = 128
n_label = 10

In [5]:
def BiRNN(x, weights, biases):
    # Prepare data shape to match 'bidirectional_rnn' function requirement
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape(batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape(batch_size, n_input)
    x = tf.unstack(x, num=n_steps, axis=1)
    
    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(n_h1, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(n_h1, forget_bias=1.0)
    
    # Get lstm cell output:
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                                     dtype=tf.float32)
    except Excpetion:
        outputs = rnn.static_birectional_rnn(lstm_fw_cell, lstm_bw_cell, x, 
                                            dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
graph = tf.Graph()
with graph.as_default():
    Xtr = tf.placeholder(tf.float32, [None, n_steps, n_input])
    Ytr = tf.placeholder(tf.float32, [None, n_label])
    
    weights = {
        # Hidden layer wights = 2*n_h1, because of fw+bw cells
        'out': tf.Variable(tf.random_normal([2*n_h1, n_label]))
    }
    
    biases = {
        'out': tf.Variable(tf.random_normal([n_label]))
    }
    
    pred = BiRNN(Xtr, weights, biases)
    
    # Define cost and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Ytr))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Ytr,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [8]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    while step*batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        feed_dict = { Xtr: batch_x, Ytr: batch_y }
        sess.run(optimizer, feed_dict=feed_dict)
        if step % display_step == 0:
            cost_, accuracy_ = sess.run([cost, accuracy], feed_dict=feed_dict)
            print ("iter: %d, Minibatch Loss=%.4f, Training Accuracy=%.4f"%
                  (step*batch_size, cost_, accuracy_))
        step += 1
    print ("Optimization Finished!")
    
    test_len = 128
    test_accuracy = sess.run(accuracy, feed_dict={
        Xtr: mnist.test.images[:test_len].reshape((-1, n_steps, n_input)),
        Ytr: mnist.test.labels[:test_len] })
    print ("Test Accuracy: %.4f"% (test_accuracy))

iter: 3200, Minibatch Loss=1.3698, Training Accuracy=0.5625
iter: 6400, Minibatch Loss=0.7079, Training Accuracy=0.7578
iter: 9600, Minibatch Loss=0.5917, Training Accuracy=0.8438
iter: 12800, Minibatch Loss=0.3400, Training Accuracy=0.9062
iter: 16000, Minibatch Loss=0.4046, Training Accuracy=0.8906
iter: 19200, Minibatch Loss=0.2292, Training Accuracy=0.9219
iter: 22400, Minibatch Loss=0.2379, Training Accuracy=0.9141
iter: 25600, Minibatch Loss=0.2199, Training Accuracy=0.9375
iter: 28800, Minibatch Loss=0.1850, Training Accuracy=0.9297
iter: 32000, Minibatch Loss=0.1298, Training Accuracy=0.9609
iter: 35200, Minibatch Loss=0.2192, Training Accuracy=0.9141
iter: 38400, Minibatch Loss=0.2761, Training Accuracy=0.9141
iter: 41600, Minibatch Loss=0.2370, Training Accuracy=0.9141
iter: 44800, Minibatch Loss=0.2119, Training Accuracy=0.9375
iter: 48000, Minibatch Loss=0.2465, Training Accuracy=0.9062
iter: 51200, Minibatch Loss=0.1354, Training Accuracy=0.9453
iter: 54400, Minibatch Loss